In [8]:
import z3
from Crypto.Util.number import bytes_to_long,long_to_bytes

```python
from Crypto.Util.number import bytes_to_long

p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
a = b'ictf{REDACTED}'

assert((p - bytes_to_long(a)) > 0 )

def devious_hash(pt):
    A = bytes_to_long(pt)
    hashed = 0
    for i in range(8):
        hashed += A
        hashed &= p
        hashed = pow(hashed,2,p)
    return hashed
print(devious_hash(a))

#output is 5529146955026504451007428379478468505767752969729288045439800587429289159437142456530335848325725612485348038928038452092595039114633083434109252927938183950
```

In [3]:
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
a = b'ictf{REDACTED}'
o = 5529146955026504451007428379478468505767752969729288045439800587429289159437142456530335848325725612485348038928038452092595039114633083434109252927938183950
assert((p - bytes_to_long(a)) > 0 )

def devious_hash(pt):
    A = bytes_to_long(pt)
    hashed = 0
    for i in range(8):
        hashed += A
        hashed &= p
        hashed = pow(hashed,2,p)
    return hashed

In [35]:
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
o = devious_hash(a)

In [36]:
s = z3.Solver()

bv = p.bit_length()
assert p < 2**bv
z_p = z3.BitVecVal(p,bv)
z_o = z3.BitVecVal(o,bv)

z_A = z3.BitVec("flag",bv) 
z_hash = z3.BitVecVal(0,bv)

for i in range(8):
    z_hash = z_hash + z_A
    # z_hash = z_hash & z_p
    z_hash = z_hash * z_hash
    z_hash = z3.URem(z_hash,z_p)

s.add(z_A < z_p)
s.add(z_hash == z_o)


In [37]:
s.check()

unsat

# Ideas

1. The last iteration contain our flag
```python3
A = bytes_to_long(pt)
hashed = 0
for i in range(8):
    hashed += A
    hashed &= p
    hashed = pow(hashed,2,p)
```

which means that

```python3
A = bytes_to_long(pt)
hashed = rnd
hashed += A
hashed &= p
hashed = pow(hashed,2,p)
```
2. However, we also start with 0

```
hashed += A
hashed &= p
hashed = pow(hashed,2,p) 

=> 1. hashed = (A*2) % p
=> 2. hashed = ((A+((A&p)^2))&p)^2 % p

```

# Mersenne Prime

In [ ]:
# chatgpt told me p is actually a persenne prime :D (2^p - 1 is prime but also 2^n - 1 with & is the same as MOD 2^n)
assert p.bit_count() == p.bit_length()

In [42]:
o = 5529146955026504451007428379478468505767752969729288045439800587429289159437142456530335848325725612485348038928038452092595039114633083434109252927938183950
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151

In [47]:
s = z3.Solver()

z_p = z3.IntVal(p)
z_p1 = z3.IntVal(p+1)
z_o = z3.IntVal(o)

z_A = z3.Int("flag") 
z_hash = z3.IntVal(0)

for i in range(8):
    z_hash = z_hash + z_A
    z_hash = z_hash % z_p1
    z_hash = z_hash * z_hash
    z_hash = z_hash % z_p

s.add(z_hash == z_o)
s.check()

unknown

In [48]:
s.reason_unknown()

'interrupted from keyboard'

# Some guessing
((0+A % p+1)^2 % p)

((0+A & p)^2 % p)

((0+A & p)^2 % p)


0 <= (0+A & p)^2 <= p-1

0 <= A & p       <= p

# Round assertions


# First round

A^2 % p

# Second round

((A^2 + A) & p) % p 

If we do tonkshel, we get

(A^2 + A) & p 



# Are the implementations the same?

In [ ]:
import z3
bv = p.bit_length() * 2
z_p = z3.BitVecVal(p,bv)

z_A1 = z3.BitVec("x1",bv) 
z_A2 = z3.BitVec("x2",bv) 

z_hash1 = z3.BitVecVal(0,bv)
z_hash2 = z3.BitVecVal(0,bv)

for i in range(2):
    z_hash1 = z_hash1 + z_A1
    z_hash1 = z_hash1 * z_hash1
    z_hash1 = z3.URem(z_hash1,z_p)
    
    z_hash2 = z_hash2 + z_A2
    z_hash2 = z_hash2 & z_p
    z_hash2 = z_hash2 * z_hash2
    z_hash2 = z3.URem(z_hash2,z_p)

z3.prove(
    z3.Implies(z_A1 == z_A2,z_hash1 == z_hash2)
)

# does not return any time soon

In [2]:
bv = p.bit_length()
z_p = z3.BitVecVal(p,bv)

z_A1 = z3.BitVec("x1",bv) 
z_A2 = z3.BitVec("x2",bv) 

z_hash1 = z3.BitVecVal(0,bv)
z_hash2 = z3.BitVecVal(0,bv)

for i in range(2):
    z_hash1 = z_hash1 + z_A1
    z_hash1 = z_hash1 * z_hash1
    
    z_hash2 = z_hash2 + z_A2
    z_hash2 = z_hash2 * z_hash2
    z_hash2 = z3.URem(z_hash2,z_p)

z_hash1 = z3.URem(z_hash1,z_p)

z3.prove(
    z3.Implies(z_A1 == z_A2,z_hash1 == z_hash2)
)

NameError: name 'z3' is not defined

# Therefore

$h_0 = 0$

$h_i = (h_{i-1} + A)^2 \mod p$

$h_1 = A^2, h_2 = (A^2 + A)^2, h_3 = (((A^2 + A)^2) + A)^2$

In [ ]:
z_p = z3.IntVal(p)
z_A1 = z3.Int("x1",bv) 
z_hash1 = z3.IntVal(0)

for i in range(8):
    z_hash1 = z_hash1 + z_A1
    z_hash1 = z_hash1 * z_hash1
    
z_hash1 = z3.URem(z_hash1,z_p)    
z_hash1

simplified = z3.simplify(z_hash1)
simplified

bvurem_i((((((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1)*
           (((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1) +
           x1)*
          ((((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1)*
           (((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1) +
           x1) +
          x1)*
         (((((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1)*
           (((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1) +
           x1)*
          ((((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1) +
            x1)*
           (((((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1)*
             (((x1*x1 + x1)*(x1*x1 + x1) + x1)*
              ((x1*x1 + x1)*(x1*x1 + x1) + x1) +
              x1) +
             x1)*
            ((((x1*x1 + x1)*(x1*x1 + x1) +

# What is the problem? 

We calcualte $\mod p+1$ with & and $\mod p$ with %

In [4]:
p & p,p % p

(6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151,
 0)

In [5]:
(p+1) & p,(p+1) % p

(0, 1)

In [6]:
(p+p-1) & p,(p+p-1) % p

(6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057149,
 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057150)

To have the same results, we need to substract 1 or 0 in all possible $2^8$ ways as we do not know when the overlap happend. This will work as the flag in long form is always smaller than p s.t. the difference is always $1$!. 

Why substract? We have to reverse the operation, hence we need to take 1 off.

# Solve with sage

In [13]:
from sage.all import * 
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
target = 5529146955026504451007428379478468505767752969729288045439800587429289159437142456530335848325725612485348038928038452092595039114633083434109252927938183950
Fp = GF(p)

for testset in range(256):
    
    """
    powerset generation in the form of
    [0,0,0,0,0,0,0,0]
    [0,0,0,0,0,0,0,1]
    [0,0,0,0,0,0,1,0]
    [0,0,0,0,0,0,1,1]
    """
    
    print(testset)
    bits = [Fp(-int(b)) for b in f"{testset:08b}"]
    P = PolynomialRing(Fp,"A")
    A = P.gen(0)

    hashed = Fp(0)
    for bit in bits:
        hashed += A 
        hashed += bit 
        hashed = (hashed)**2
    
    f = hashed - target
    for root in f.roots(multiplicities=false):
        candidate = long_to_bytes(int(root))
        if b'ictf' in candidate:
            print("Possible flag:", candidate)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Possible flag: b'____ictf{devious_mersenne_prime_makes_root_finding_easy_actually}'
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


32
Possible flag: b'____ictf{devious_mersenne_prime_makes_root_finding_easy_actually}'